In [ ]:
from util.domainUtil import getIp,getSanitzedApps, generateTotalCount
import util.countryFileUtil
import importlib
from util.util import mergeDicts
importlib.reload(util.util)
from util.util import  loadAllData, getAllDataDataset, get_verified_dataset
import tldextract


In [ ]:
util.countryFileUtil.loadIPs()
#util.countryFileUtil.getCountries(appListSanitized[0])

In [ ]:
def getMapFromDataset(dataset):
    result = {}
    for i in range(0, len(dataset)):
        stats = util.countryFileUtil.getCountries(dataset[i])
        result = mergeDicts(result, stats)
    return result

In [ ]:
# TODO: set the path to your result folder
# set the first argument to the base path of the result folder, second argument is th folder name, third argument a mapping file - that tells for apps in multiple sub datasets which app to include

verified = get_verified_dataset("/", "/2023_04_06/", "/verified_dataset/result.json")
allDataVerified = getAllDataDataset(verified)
appListSanitizedVerified = getSanitzedApps(allDataVerified, True, False) # Subdomains and Count subdomains, set to false
mapVer = getMapFromDataset(appListSanitizedVerified)

%store mapVer
%store appListSanitizedVerified
%store  allDataVerified


In [ ]:
# TODO: set the path to load your data
generalDataset = loadAllData("/gp_2022/", True)
allDatageneral = getAllDataDataset(generalDataset)
appListSanitizedGeneral = getSanitzedApps(allDatageneral, True, False) # Subdomains and Count subdomains, set to false
mapGeneral = getMapFromDataset(appListSanitizedGeneral)
%store mapGeneral
%store appListSanitizedGeneral
%store  allDatageneral


In [ ]:
%store allDatageneral


%store appListSanitizedGeneral




In [ ]:
%store -r allDataVerified
%store  -r allDatageneral


%store -r appListSanitizedVerified
%store -r appListSanitizedGeneral


%store -r mapGeneral
%store -r mapVer


In [ ]:
devLocationGeneral = {"SG": 35,
    "US": 300,
    "DE": 98,
    "SC": 1,
    "PT": 2,
    "RU": 11,
    "CY": 52,
    "AT": 65,
    "AE": 12,
    "CN": 9,
    "HK": 37,
    "ID": 1,
    "FI": 11,
    "BR": 3,
    "FR": 45,
    "NL": 16,
    "AU": 4,
    "IE": 9,
    "JP": 16,
    "KR": 6,
    "GB": 24,
    "SK": 2,
    "IN": 6,
    "TR": 16,
    "PK": 7,
    "LU": 4,
    "IL": 40,
    "CH": 12,
    "MD": 1,
    "SE": 11,
    "VN": 33,
    "VG": 7,
    "GR": 1,
    "DK": 6,
    "RO": 2,
    "UA": 2,
    None: 12,
    "IT": 3,
    "LV": 2,
    "BD": 1,
    "EE": 2,
    "PH": 4,
    "NO": 2,
    "BY": 2,
    "CA": 7,
    "CZ": 1,
    "ES": 8,
    "LT": 2,
    "CR": 1,
    "HU": 1,
    "AG": 1,
    "WS": 1,
    "NZ": 1,
    "MT": 1,
    "RS": 2,
    "PL": 1}

In [ ]:
data = []
overallMap = {}


overallMap = mergeDicts(mapVer, overallMap)
overallMap = mergeDicts(mapGeneral, overallMap)
overallMap = mergeDicts(devLocationGeneral, overallMap)


In [ ]:
sortedMap = {k: v for k, v in sorted(overallMap.items(),reverse=True, key=lambda item: item[1])}

In [ ]:
for key in sortedMap.keys():
    row = []
    row.append(key)
    row.append(mapVer.get(key,0))

    row.append(mapGeneral.get(key,0))
    row.append(devLocationGeneral.get(key,0))

    data.append(row)

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns=['Country', 'IoT-VER' ,'IoT-2018', 'IoT-2021', 'IoT-2022','Dev IoT-2022' ,'GP-2022', 'Dev GP-2022'])

In [ ]:
euCodes = ["EU","LI","AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"]
euDF = df.loc[df['Country'].isin(euCodes)]

In [ ]:
withoutEU = df.drop(df[df['Country'].isin(euCodes)].index)

In [ ]:
asianCountry = ['SG', 'HK', 'JP', 'KR', 'TW', 'VN', 'IN', 'PK', 'ID', 'PH', 'BD', 'MY', 'TH', 'IR']
asianDF = df.loc[df['Country'].isin(asianCountry)]

In [ ]:
withouAsianAndEU = withoutEU.drop(withoutEU[withoutEU['Country'].isin(asianCountry)].index)

In [ ]:
otherCodes = ["CL","MK","IM","NG","UG",'CO','CA', None, 'TN','TR', 'MU','CH', 'ZA','TZ', 'AU', 'AE','ZA', 'SC', 'NO', 'IL', 'VG', 'BR', 'IS', 'PA', 'RS', 'UA', 'ID', 'BY','RS', 'NZ', 'BZ', 'PA', 'WS', 'CR', 'EG', 'AR', 'SA', 'MX', 'AZ', 'AG', 'MD']
otherDF = df.loc[df['Country'].isin(otherCodes)]

In [ ]:
withoutOthers = withouAsianAndEU.drop(withouAsianAndEU[withouAsianAndEU['Country'].isin(otherCodes)].index)

In [ ]:
def createRowFromDF(df, newKey):
    row = {}
    summed = df.sum()
    row['Country'] = (newKey)
    row['IoT-VER'] = summed['IoT-VER']

    row['GP-2022'] = summed['GP-2022']
    row['Dev GP-2022'] = summed['Dev GP-2022']

    return row


In [ ]:
otherRow = createRowFromDF(otherDF, 'other')
euRow = createRowFromDF(euDF, 'EU')
asiaRow = createRowFromDF(asianDF, 'Asia')

In [ ]:
overallDF = withoutOthers.append(otherRow, ignore_index=True).append(euRow, ignore_index=True).append(asiaRow, ignore_index=True)

In [ ]:
overallDF

In [ ]:
overallDF = overallDF.set_index('Country')

In [ ]:
overallDF = overallDF.reindex(['US', 'CN', 'EU', 'Asia', 'GB', "RU", "other"])

In [ ]:
overallDF

In [ ]:
print(overallDF.to_latex())

In [ ]:
overallPercentage = overallDF.div(overallDF.sum(axis=0), axis=1)

In [ ]:
overallPercentage.to_csv('countries_processed_ccs.csv')

In [ ]:
df.to_csv('countries_ccs.csv')

In [ ]:
withoutOthers

In [ ]:
df

In [ ]:
overallPercentage

In [ ]:
overallPercentage = overallPercentage.style.format("{:.2%}")

In [ ]:
print(overallPercentage.to_latex())

In [ ]:
overallPercentage


In [ ]:
summed = overallDF.sum(axis=0)

In [ ]:
overallDF

In [ ]:
data = []

for row in (overallDF.index):
    current = []
    for column in overallDF.columns:

        calc = int(overallDF.loc[row, column])/int(summed[column])
        calc = calc * 100
        #overallDF.at[row, column] = f"{overallDF.loc[row, column]} ({calc}%)"
        current.append( f"{overallDF.loc[row, column]:,} ({calc:.2f}%)")

    data.append(current)

In [ ]:
df_final = pd.DataFrame(data, index=overallDF.index, columns=overallDF.columns)

In [ ]:
df_final

In [ ]:
print(df_final.to_latex())